# Light FM 

Light FM is an hybrid recommender model that can be used to learn user - item representations and preferences.

In this notebook we are going to try and use it on some real world data.

In [3]:
# Get the data
data_destination_path = '~/Documents/github/MachineLearning/Udemy-PRSFB/LightFM/data'

In [ ]:
import os

os.system(f'mkdir {data_destination_path}')
os.system(f'curl -L -o {data_destination_path}/archive.zip  https://www.kaggle.com/api/v1/datasets/download/hernan4444/anime-recommendation-database-2020')
os.system(f'unzip -o -d {data_destination_path} {data_destination_path}/archive.zip')
os.system(f'rm {data_destination_path}/archive.zip')

In [4]:
import pandas as pd

df = pd.read_csv(data_destination_path + '/rating_complete.csv')

In [5]:
df.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df.to_numpy(), test_size=.25)
print(train[:5])

[[281610     32      8]
 [260583    202      8]
 [262165    226      4]
 [167953  15809      7]
 [ 48795  20767      9]]


In [10]:
from scipy import sparse

train_sp  = sparse.csr_matrix(train)

In [11]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr', no_components=1024)
model.fit(train_sp, epochs=1, num_threads=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

MemoryError: Unable to allocate 330. GiB for an array with shape (43224958, 1024) and data type float64